In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Diabetes/diabetic_data.csv')

In [3]:
dataset.replace('?', np.nan, inplace=True)

In [4]:
dataset.drop(['weight','payer_code','medical_specialty'],axis=1,inplace=True)

In [5]:
dataset.dropna(inplace=True)

In [6]:
dataset.drop(['encounter_id','patient_nbr','admission_type_id','discharge_disposition_id','admission_source_id'],axis=1,inplace=True)

In [7]:
dataset

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,Caucasian,Female,[10-20),3,59,0,18,0,0,0,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),2,11,5,13,2,0,1,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),2,44,1,16,0,0,0,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,51,0,8,0,0,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,Caucasian,Male,[50-60),3,31,6,16,0,0,0,...,No,Steady,No,No,No,No,No,No,Yes,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),3,51,0,16,0,0,0,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,AfricanAmerican,Female,[80-90),5,33,3,18,0,0,1,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,Caucasian,Male,[70-80),1,53,0,9,1,0,0,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,Caucasian,Female,[80-90),10,45,2,21,0,0,1,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [8]:
dataset.loc[dataset.age== '[0-10)','age'] = 0;
dataset.loc[dataset.age== '[10-20)','age'] = 10;
dataset.loc[dataset.age== '[20-30)','age'] = 20;
dataset.loc[dataset.age== '[30-40)','age'] = 30;
dataset.loc[dataset.age== '[40-50)','age'] = 40;
dataset.loc[dataset.age== '[50-60)','age'] = 50;
dataset.loc[dataset.age== '[60-70)','age'] = 60;
dataset.loc[dataset.age== '[70-80)','age'] = 70;
dataset.loc[dataset.age== '[80-90)','age'] = 80;
dataset.loc[dataset.age== '[90-100)','age'] = 90;
dataset.age = dataset.age.astype(np.int32)


dataset.loc[dataset.max_glu_serum== 'None','max_glu_serum'] = 0;
dataset.loc[dataset.max_glu_serum== 'Norm','max_glu_serum'] = 100;
dataset.loc[dataset.max_glu_serum== '>200','max_glu_serum'] = 200;
dataset.loc[dataset.max_glu_serum== '>300','max_glu_serum'] = 300;
dataset.max_glu_serum = dataset.max_glu_serum.astype(np.int32)


dataset.loc[dataset.A1Cresult== 'None','A1Cresult'] = 0;
dataset.loc[dataset.A1Cresult== 'Norm','A1Cresult'] = 5;
dataset.loc[dataset.A1Cresult== '>7','A1Cresult'] = 7;
dataset.loc[dataset.A1Cresult== '>8','A1Cresult'] = 8;
dataset.A1Cresult = dataset.A1Cresult.astype(np.int32)


dataset.loc[dataset.change== 'No','change'] = 0;
dataset.loc[dataset.change== 'Ch','change'] = 1;
dataset.change = dataset.change.astype(np.int8)



dataset.loc[dataset.diabetesMed== 'No','diabetesMed'] = 0;
dataset.loc[dataset.diabetesMed== 'Yes','diabetesMed'] = 1;
dataset.diabetesMed = dataset.diabetesMed.astype(np.int8)


medications = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"]

for med in medications:
    dataset.loc[dataset[med] == 'No', med] = -20;
    dataset.loc[dataset[med] == 'Down', med] = -10;
    dataset.loc[dataset[med] == 'Steady', med] = 0;
    dataset.loc[dataset[med] == 'Up', med] = 10;
    dataset[med] = dataset[med].astype(np.int32)
    

categoricals = ['race', 'gender', 'diag_1', 'diag_2', 'diag_3']



for c in categoricals:
    dataset[c] = pd.Categorical(dataset[c]).codes

In [9]:
dataset.loc[dataset.readmitted != 'NO','readmitted'] = 0
dataset.loc[dataset.readmitted == 'NO','readmitted'] = 1

In [10]:
dataset.readmitted = dataset.readmitted.astype(np.int8)

In [11]:
X, y = dataset.drop('readmitted', axis=1), dataset['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [12]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

[WARNING] [2023-06-13 18:18:13,946:Client-EnsembleBuilder] No runs were available to build an ensemble from


In [13]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.6295586195643306

In [14]:
import pickle
with open('./models/diabetes_automl2.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [12]:
import pickle
with open('./models/diabetes_automl2.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())
y_hat = automl.predict(X_test.values)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.6295586195643306

# RANDOM FOREST

In [45]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [46]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.6187076772456556

# SVC

In [15]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=2, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


GridSearchCV(cv=2, estimator=SVC(random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10]}, verbose=3)

In [16]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.584155992494085

# LIME

In [13]:
import lime
import lime.lime_tabular
import tqdm 

In [14]:
continuous_features = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['Readmitted', 'Not Readmitted'], categorical_features=categorical_features, discretize_continuous=True)

In [15]:
frac = 0.05
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, train_size=frac, random_state=42, stratify=y_test)

test_x = X_test_sample.values
test_y = y_test_sample.values

In [16]:
exp_fn = lambda i: explainer.explain_instance(test_x[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [17]:
import metrics

In [37]:
exp1 = exp_fn_wrap(test_x)
exp2 = exp_fn_wrap(test_x)

100%|██████████| 1225/1225 [1:44:28<00:00,  5.12s/it]


In [38]:
np.save('./explanations/diabetes1.npy', exp1)
np.save('./explanations/diabetes2.npy', exp2)

In [18]:
exp1 = np.load('./explanations/diabetes1.npy')
exp2 = np.load('./explanations/diabetes2.npy')

In [39]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [41]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, test_y)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [42]:
i, s, sb

((100.0, 0, 1225), (0, 1225, 1500625, 0.0), (461, 1225))

In [43]:
X_test_norm = metrics.normalize_test(X_train, X_test_sample)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [44]:
sim

0.1566047226055056

In [45]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(test_x))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, test_y[i], 1)
    example = metrics.FeatureAttribution(automl, test_x[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 1225/1225 [5:10:59<00:00, 15.23s/it] 


In [46]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.013725995349142017
8.973877551020408
0.0
0.005262040497850872
9.0
0.0


In [19]:
trust = metrics.calc_trust_score(automl, test_x, exp1, 3, X_train.columns.tolist())

100%|██████████| 1225/1225 [15:31:04<00:00, 45.60s/it]  


In [20]:
trust

0.21170068027210887